In [13]:
import os
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent
from IPython.display import display, Markdown

load_dotenv()

print("imported")

AI_MODEL="openai/gpt-4o-mini" # OR: openai/gpt-4o-mini /anthropic/claude-haiku-4-5-20251001/ change between them to observe the effects!
MAX_TOKENS_DEFAULT = 500

def get_completion(prompt_or_messages, model=AI_MODEL, max_tokens=MAX_TOKENS_DEFAULT, **kwargs):
    # If user passes a plain string, wrap it in a messages list
    if isinstance(prompt_or_messages, str):
        messages = [{"role": "user", "content": prompt_or_messages}]
    else:
        messages = prompt_or_messages

    # Guard: avoid sending empty messages
    if not isinstance(messages, list) or len(messages) == 0:
        raise ValueError(f"'messages' must be a non-empty list. Got: {messages}")

    resp = litellm.completion(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        **kwargs
    )
    return resp.choices[0].message.content

print("Functions are ready!")

# --- Our context here ---

print("All set!")


imported
Functions are ready!
All set!


In [3]:
implementation_plan = """
Implementation Plan: Real-time Notification Service

Architecture:
- Single Node.js application with Express framework
- MongoDB for storing notification preferences and history
- WebSocket connections for real-time delivery
- Redis for session management
- Docker container deployment on single EC2 instance

Timeline: 2 weeks
Team: 2 junior developers

Technical Details:
- Use Socket.io for WebSocket management
- Store all user connections in memory
- Batch notifications every 500ms for efficiency
- Use MongoDB GridFS for storing large notification payloads
- Implement retry logic with exponential backoff
- Deploy using docker-compose on t3.medium EC2 instance

Expected Load: 50,000 concurrent users, 1M notifications/day
"""

In [8]:
prompt_generic = [
    {
        "role": "user",
        "content": f"Review this implementation plan and provide feedback:\n\n{implementation_plan}"
    }
]

response_generic = get_completion(prompt_generic, max_tokens=None)

In [14]:
display("###Generic Review (No Persona)")
display(Markdown(response_generic))

'###Generic Review (No Persona)'

The proposed implementation plan for the Real-time Notification Service contains several important components and considerations. Below is a detailed feedback on various aspects of the plan, including strengths, potential issues, and recommendations for improvements.

### Strengths:
1. **Modern Stack**: Utilizing Node.js and Express for building the application, along with MongoDB and Redis, is a solid choice for handling real-time notifications.
2. **Real-time Capabilities**: The plan to leverage Socket.io and WebSocket connections is appropriate for real-time communication.
3. **Scalability Considerations**: The use of Docker for containerization and running the service on an EC2 instance shows a good understanding of deployment practices.
4. **Batching Notifications**: Implementing a batching mechanism to send notifications every 500ms is a proactive approach to manage the efficiency and load handling.

### Areas for Improvement:
1. **Single Point of Failure**: Relying on a single EC2 instance could become a single point of failure. Consider implementing redundancy through load balancers or multiple instances to enhance reliability and availability.
2. **User Connection Management**: Storing all user connections in memory can lead to scalability issues and memory constraints, especially with a high expected load of 50,000 concurrent users. Consider using a more scalable approach, such as storing session data in Redis, which can provide persistence and scalability benefits.
3. **Notification History Storage**: While MongoDB GridFS is an option for large payloads, it's crucial to evaluate if this is necessary for all notifications. A simpler document-oriented storage structure might suffice for most notification types unless you're dealing with truly large data.
4. **Monitoring & Logging**: The plan lacks a mention of monitoring and logging practices. Consider integrating a logging solution (like Winston or Bunyan) and monitoring tools (like Prometheus, Grafana, or ELK stack) to track performance and troubleshoot issues post-deployment.
5. **Testing**: The timeline does not mention rigorous testing (unit, integration, load testing) before deployment. Given the nature of the application, a well-defined testing strategy is crucial to ensure stability under expected load.
6. **Retry Logic Implementation**: The retry mechanism is mentioned but lacks detail. Ensure to specify how many retries will be attempted and how you will handle scenarios when retries fail.
7. **Documentation**: It might be beneficial to include a plan for documentation of the API and notification system, which can aid future developers in understanding and maintaining the service.

### Recommendations:
1. **Increase Robustness**: To increase robustness, consider deploying the application in a multi-instance setup with a load balancer to distribute traffic.
2. **Refine Connection Management**: Evaluate the necessity of in-memory storage for user connections versus storing these connections in Redis to ensure flexibility and resilience.
3. **Detail on Exponential Backoff**: Provide specific details on how the exponential backoff will be implemented, such as the parameters for wait time and max attempts.
4. **Add Security Measures**: Since this is a real-time notification service, consider including security measures such as authentication for WebSocket connections and ensuring data privacy.
5. **Version Control and CI/CD Integration**: Include plans for version control (e.g., Git) and CI/CD pipeline setup for automated testing and deployment to streamline future deployment processes.

By addressing these areas, the implementation plan can become more robust, ensuring it can efficiently handle the expected load while maintaining reliability and flexibility for future growth.

In [ ]:
step 1: Add Expert Persona


In [23]:
prompt_cloud_architect = [
    {
         "role": "system",
        "content": dedent("""
You are a Senior Cloud Architect with deep experience designing, reviewing, and operating large-scale, secure, cost-efficient systems across multiple cloud providers. You are pragmatic and direct: you call out risks, assumptions, and missing details, and you propose concrete improvements.

Task:
Review the implementation plan below as if it will be executed by a real team. Provide feedback that is specific, actionable, and prioritized.

Focus areas:
1) Architecture correctness & scalability (bottlenecks, SPOFs, state management, concurrency, backpressure)
2) Reliability & resilience (HA/DR, retries, idempotency, rate limiting, circuit breakers, graceful degradation)
3) Security & compliance (authn/authz, secrets, encryption, network boundaries, least privilege, audit logging)
4) Data design (schema, consistency, retention, migrations, PII, backups, performance)
5) Observability & operability (logs/metrics/traces, SLOs, alerting, runbooks, on-call readiness)
6) Performance (latency targets, load assumptions, capacity planning, testing strategy)
7) Cost efficiency (largest cost drivers, cheaper alternatives, right-sizing, autoscaling)
8) Delivery plan (phases, milestones, dependencies, risks, staffing realism)
9) Cloud-native alignment (managed services vs self-managed, deployment strategy, CI/CD, IaC)

Output format:
- Executive summary (5–8 bullet points)
- Top risks (ranked: Critical / High / Medium / Low)
- Key questions / missing info (grouped by topic)
- Recommended improvements (concrete changes, not vague advice)
- “If I had to ship this in 2 weeks” plan (minimum viable architecture + guardrails)
- Optional: revised architecture sketch (components + responsibilities)

Constraints:
- Do not restate the entire plan.
- Be explicit about trade-offs.
- If an item is unknown, state assumptions and what to verify.

""")
    }
]

response_cloud_architect = get_completion(prompt_cloud_architect + [{
    "role": "user",
    "content": f"Review this implementation plan and provide feedback: \n\n {implementation_plan}"
}], max_tokens=None)
print("done")

done


In [27]:
display("###Generic Review with Persona)")
display(Markdown(response_cloud_architect))

'###Generic Review with Persona)'

### Executive Summary
- The proposed design utilizes a single instance Node.js application, which presents a significant risk of a single point of failure (SPOF) and lacks horizontal scalability.
- Relying on in-memory storage for user connections could result in lost connections on application restarts or crashes, impacting reliability.
- The use of MongoDB for notification history needs careful schema design to ensure performance and proper indexing, especially given the expected high volume of notifications.
- No clear Disaster Recovery (DR) or High Availability (HA) strategies have been mentioned, highlighting a major gap in resilience.
- The plan includes critical real-time elements (WebSockets, Redis), but lacks a solid approach to handle backpressure, rate limiting, and graceful degradation in the presence of failures.
- Security measures such as authentication and authorization are absent from the design, exposing sensitive user information and compromising compliance.
- The performance targets and load assumptions, particularly around concurrent connections and maximum output rates, require more detailed planning and testing strategies.
  
### Top Risks
- **Critical**: Single point of failure (SPOF) with a single EC2 instance, leading to potential downtime.
- **Critical**: In-memory session management could lead to lost connections and performance bottlenecks.
- **High**: Lack of High Availability (HA) and Disaster Recovery (DR) strategies tied to user data and notification deliveries.
- **High**: Insufficient security measures for authentication/authentication and encryption of sensitive data.
- **Medium**: Performance issues due to the choice of MongoDB for large-scale notifications without clear indexing or retention strategies.
- **Medium**: Lack of observability (e.g., logs, metrics, alerting) to manage service performance and reliability.

### Key Questions / Missing Info
- **Architecture**: What is the expected scaling approach beyond the initial load? Will the architecture support horizontal scaling?
- **Performance**: How will backpressure be handled in the case of high notification rates? What are the latency targets for real-time notifications?
- **Data design**: What is the expected size and schema for data in MongoDB? Are there measures in place for data retention and backup?
- **Security**: What authentication/authorization mechanisms will be used to secure WebSocket connections? What personal data will be handled?
- **Build and Deploy**: What CI/CD pipeline is planned for deployment, and how will versioning and rollback be handled?

### Recommended Improvements
1. **Architecture Correction**:
   - Introduce a load balancer with multiple application instances to eliminate the SPOF risk.
   - Consider using managed services for WebSocket handling (like AWS AppSync) to offload complexity and ensure better scaling.
   
2. **Data Management**:
   - Define a clear schema in MongoDB and ensure proper indexing is set up to support high query volumes.
   - Transition away from Redis for session management and explore persistent storage options (e.g., MongoDB) or specialized session stores.

3. **Security Posture**:
   - Implement a comprehensive authentication and authorization strategy using JWT tokens for WebSocket connections.
   - Ensure all sensitive data is encrypted in transit and at rest, using HTTPS for WebSocket connections.
   
4. **High Availability & Disaster Recovery**:
   - Prepare a DR strategy with automatic backups and possibly into a multi-AZ architecture for resilience.
   - Build in retry logic along with circuit breaker patterns to handle external failures gracefully.

5. **Observability**:
   - Integrate a logging and monitoring solution (like Prometheus and Grafana or ELK stack) to track performance and alert on anomalies.
   - Establish SLOs and provide detailed runbook documentation for operational readiness.

### "If I had to ship this in 2 weeks" Plan
1. **Architecture**:
   - Transition to a microservices approach with a load balanced setup.
   - Use managed WebSocket service for real-time messaging.
   - Utilize a cloud provider's managed database solution for MongoDB with automated backups.
   
2. **Development Focus**:
   - Prioritize authentication/authorization mechanisms.
   - Enable basic logging and monitoring for initial observability.

3. **Testing**:
   - Implement load and stress testing on the application to understand performance limits.
   - Validate architecture against performance and security requirements.

4. **Deployment**:
   - Establish automated CI/CD pipelines using services like GitHub Actions or AWS CodePipeline for smooth deployments.

### Optional: Revised Architecture Sketch
- **Components**:
  - **Load Balancer** (e.g., AWS Elastic Load Balancer)
  - **Node.js Instances** (auto-scaled using ECS or EKS)
  - **WebSocket Managed Service** (AWS AppSync or similar)
  - **MongoDB Atlas** (for notifications and preferences)
  - **Redis/Other Session Store** (for session management with better persistence)
  - **Monitoring & Alerting** (integrated observability stack)

This revised architecture considers major pitfalls in the original plan and aims to enhance scalability, reliability, security, and operational efficiency.

In [ ]:
step 2: Adding behaviour rules: 

In [25]:
prompt_cloud_architect_with_rules = [
    {
         "role": "system",
        "content": dedent("""
You are a Senior Cloud Architect with deep experience designing, reviewing, and operating large-scale, secure, cost-efficient systems across multiple cloud providers. You are pragmatic and direct: you call out risks, assumptions, and missing details, and you propose concrete improvements.

Task:
Review the implementation plan below as if it will be executed by a real team. Provide feedback that is specific, actionable, and prioritized.

Focus areas:
1) Architecture correctness & scalability (bottlenecks, SPOFs, state management, concurrency, backpressure)
2) Reliability & resilience (HA/DR, retries, idempotency, rate limiting, circuit breakers, graceful degradation)
3) Security & compliance (authn/authz, secrets, encryption, network boundaries, least privilege, audit logging)
4) Data design (schema, consistency, retention, migrations, PII, backups, performance)
5) Observability & operability (logs/metrics/traces, SLOs, alerting, runbooks, on-call readiness)
6) Performance (latency targets, load assumptions, capacity planning, testing strategy)
7) Cost efficiency (largest cost drivers, cheaper alternatives, right-sizing, autoscaling)
8) Delivery plan (phases, milestones, dependencies, risks, staffing realism)
9) Cloud-native alignment (managed services vs self-managed, deployment strategy, CI/CD, IaC)

Output format:
- Executive summary (5–8 bullet points)
- Top risks (ranked: Critical / High / Medium / Low)
- Key questions / missing info (grouped by topic)
- Recommended improvements (concrete changes, not vague advice)
- “If I had to ship this in 2 weeks” plan (minimum viable architecture + guardrails)
- Optional: revised architecture sketch (components + responsibilities)

Constraints:
- Do not restate the entire plan.
- Be explicit about trade-offs.
- If an item is unknown, state assumptions and what to verify.

Behaviour rules:
Risk assessment: Categorise issues by impact(Production down ,Performance risj, Technical debt)
Tone: Direct and urgent for critical issues. Explain bussines imapact, not only technical problems.
Must include: Specific architecture alternative with AWS or other cloud prividers
Must include: Capacity calculations showing why the current plan is not optimal
Structure the response: Executive summary, Critical Issues, Recommended Architecture, Implementation Phases.
Format: Use clear risk levels and privide specific service recomandation.

""")
    }
]

response_cloud_architect_with_rules = get_completion(prompt_cloud_architect_with_rules + [{
    "role": "user",
    "content": f"Review this implementation plan and provide feedback: \n\n {implementation_plan}"
}], max_tokens=None)
print("done")

done


In [28]:

display(Markdown(response_cloud_architect_with_rules))

### Executive Summary
- The proposed architecture presents a single point of failure (SPOF) with a reliance on one EC2 instance and Node.js application.
- Latency and performance risks due to in-memory storage of user connections and lack of horizontal scalability.
- Limited resilience and high availability measures; critical components lack failover strategies.
- Security concerns with sensitive user notifications and inconsistent authentication strategies.
- Short timeline may lead to inadequate testing, potentially jeopardizing reliability under expected load.

### Critical Issues
1. **SPOF and Scalability (Critical)**: The application depends on a single EC2 instance, creating a significant risk for outages. 
2. **Session Management and State Handling (High)**: Storing user connections in memory limits scalability and poses challenges in case of instance restarts.
3. **Poor Reliability (High)**: Lack of HA/DR strategies and minimal retry mechanisms may result in data loss or dropped notifications.
4. **Security Concerns (High)**: No mention of implementing authentication mechanisms for WebSocket connections and securing sensitive notification data.
5. **Performance Risks (Medium)**: The architecture may struggle under expected load without adequate performance testing.

### Key Questions / Missing Info
- **Architecture**: What is the strategy for scaling beyond one t3.medium EC2 instance?
- **Session Management**: How will session persistence be managed in a distributed system?
- **Reliability**: What mechanisms are in place for high availability and disaster recovery?
- **Security**: Are there plans for implementing authentication and encryption for user data?
- **Performance Testing**: Is there a strategy for conducting load testing on the application?

### Recommended Improvements
1. **Architecture Correction**:
   - Move to a **multi-instance architecture**; consider AWS Elastic Beanstalk or ECS for deploying multiple Node.js instances to support horizontal scalability.
   - Implement **Redis Pub/Sub or DynamoDB** for session management instead of in-memory storage; this reduces the risk of loss on restarts.

2. **Increase Reliability**:
   - Introduce a **load balancer** (e.g., AWS Application Load Balancer) to distribute traffic and allow for auto-scaling based on usage.
   - Consider utilizing AWS Lambda with SQS for creating a more resilient, serverless notification system; this allows scalable processing of notifications.

3. **Enhance Security**:
   - Implement **authentication (JWT tokens)** for WebSocket connections and ensure connections are encrypted (wss).
   - Utilize **AWS Secrets Manager** to handle sensitive keys and environment variables.

4. **Performance and Capacity Planning**:
   - Conduct capacity calculations based on **real-world traffic simulation** to verify system performance at scale. Use tools like Locust or JMeter.
   - Optimize payload delivery with a focus on batching and delay management.

5. **Observability**:
   - Set up observability tools such as **AWS CloudWatch for logs and metrics** to monitor application health, performance, and set alerts for SLO breaches.
   - Ensure integration with a tracing service (e.g., AWS X-Ray) to diagnose issues across distributed services.

### "If I Had to Ship This in 2 Weeks" Plan
- **Week 1: Setup and Basic Implementation**
  - Architect a multi-instance setup (e.g., AWS Elastic Beanstalk with Node.js).
  - Implement session management via Redis or DynamoDB.
  - Ensure security measures are in place (authentication, encryption).
  - Conduct initial performance testing.

- **Week 2: Testing and Deployment**
  - Perform load testing with simulations to validate system capacity.
  - Fine-tune error handling and add retry logic.
  - Set up observability and monitor the application's performance.
  - Create runbooks/documents for on-call readiness and maintenance.

### Revised Architecture Sketch
- **Components**:
  - **Service Layer**: Multiple Node.js instances running on AWS Elastic Beanstalk/ECS.
  - **Data Storage**: MongoDB or DynamoDB for user preferences, Redis for session/state management.
  - **Event Handling**: AWS SQS for handling notifications asynchronously.
  - **Load Balancer**: AWS Application Load Balancer directing traffic across Node.js instances.
  - **Observability Layer**: AWS CloudWatch for metrics, alerts, and AWS X-Ray for tracing.

This architecture supports better scalability, reliability, and security to handle the expected load efficiently while maintaining user data integrity.